<a href="https://colab.research.google.com/github/Vinhnv0901/student_project/blob/main/recommend_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [ ]:
!git clone "https://github.com/Vinhnv0901/student_project.git"

Cloning into 'student_project'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 40 (delta 3), reused 15 (delta 2), pack-reused 20
Unpacking objects: 100% (40/40), 76.52 MiB | 10.15 MiB/s, done.


In [ ]:
data_grade = pd.read_excel("/content/student_project/Education_dataset_V2/02.diem.xlsx")
data_inf = pd.read_excel("/content/student_project/Education_dataset_V2/01.sinhvien.xlsx")


In [ ]:
data_inf = data_inf[['mssv', ' namsinh', ' gioitinh', ' noisinh', ' lopsh', ' khoa', ' hedt', ' khoahoc', ' chuyennganh2',' tinhtrang']]
data_inf.columns = ['mssv', 'namsinh', 'gioitinh', 'noisinh', 'lopsh', 'khoa', 'hedt', 'khoahoc', 'chuyennganh2','tinhtrang']
data_grade = data_grade[['mssv', ' mamh', ' malop', ' sotc', ' namhoc', ' hocky', ' diem', ' trangthai', ' mamh_tt']]
data_grade.columns = ['mssv', 'mamh', 'malop', 'sotc', 'namhoc', 'hocky', 'diem', 'trangthai', 'mamh_tt']

###merge student table and grade table

In [ ]:
data_merge = pd.merge(data_grade, data_inf, on = 'mssv', how = 'inner')

In [ ]:
print('the number of nan value each columns:',{col: list(pd.isna(data_merge[col])).count(True) for col in data_merge.columns})

the number of nan value each columns: {'mssv': 2856, 'mamh': 2856, 'malop': 2856, 'sotc': 2856, 'namhoc': 2856, 'hocky': 2856, 'diem': 2891, 'trangthai': 2856, 'mamh_tt': 2856, 'namsinh': 2857, 'gioitinh': 2856, 'noisinh': 2856, 'lopsh': 2856, 'khoa': 2856, 'hedt': 2856, 'khoahoc': 2856, 'chuyennganh2': 2856, 'tinhtrang': 2856}


###Data cleaning

In [ ]:
data_merge = data_merge.dropna(subset = ['diem'])#xóa các dòng có giá trị nan trong cột điểm 
data_merge['diem'] = data_merge['diem'].replace(' NULL', np.nan)#chuyển đổi NUll về nan để thêm giá trị
data_merge['diem'] = data_merge['diem'].fillna(method = 'ffill')# thêm giá trị điểm có các dùng nan
data_merge = data_merge.astype(dtype = {'namhoc': np.int32, 'diem': np.float32})
data_merge = data_merge.drop(data_merge[data_merge['namhoc'] == 2012].index)#xóa các dòng năm 2012 vì dữ liệu quá ít
#khóa khoảng trắng các cột mamh, khoa, khoaMH
data_merge['mamh'] = data_merge['mamh'].str.strip()
data_merge['khoa'] = data_merge['khoa'].str.strip()
data_merge = data_merge.reset_index()

<ipython-input-7-c7a6858c2d5e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['diem'] = data_merge['diem'].replace(' NULL', np.nan)#chuyển đổi NUll về nan để thêm giá trị
<ipython-input-7-c7a6858c2d5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['diem'] = data_merge['diem'].fillna(method = 'ffill')# thêm giá trị điểm có các dùng nan


###dentify cate of course

In [ ]:
#Xác định môn học thuộc khoa nào
mamh = data_merge['mamh'].copy()
for i in range(data_merge.shape[0]):
  mamh[i] = mamh[i].translate(str.maketrans("","", '0123456789'))
a = mamh.unique() #các mã môn học đại diện cho khoa quản lý
b = ['MHKT', 'KTMT', 'HTTT', 'HTTT', 'PĐTĐH', 'BMTL', 'KHMT', 'PĐTĐH', 'HTTT', 'HTTT', 'BMAV', 'MMT&TT', 'CNPM', 'BMAV', 'HTTT', 'KTTT', 'HTTT', 'KTMT', 'CNPM', 'HTTT', 'HTTT', 'HTTT','HTTT', 'PĐTĐH', 'PĐTĐH'] # các khoa
mamh.replace(to_replace = a, value = b, inplace = True)
for i in range(data_merge.shape[0]):#các nhóm môn học có của cái đầu là 'IT' cần được điều chỉnh lại
  ma = data_merge.loc[i,'mamh']
  if ma == 'IT001' or ma == 'IT003' or ma == 'IT011' or ma == 'IT013':
    mamh[i] = 'KHMT'
  elif ma == 'IT002' or ma == 'IT008':
    mamh[i] = 'CNPM'
  elif ma == 'IT004':
    mamh[i] = 'HTTT'
  elif ma == 'IT005':
    mamh[i] = 'MMT&TT'
  elif ma == 'IT006' or ma == 'IT007' or ma =='IT010' or ma == 'IT012':
    mamh[i] = 'KTMT'
  elif ma == 'IT009':
    mamh[i] = 'PĐTĐH'
data_merge['KhoaMH'] = np.array(mamh)


In [ ]:
#Không rõ là học kỳ hè có nên xóa hay không vì nó chỉ chiếm 2 phần trăm trong dataset nên tạp thời xóa ở đây thay vì xóa từ đầu????????????????????????????????????????????
data_merge = data_merge.drop(data_merge[data_merge['hocky'] == 3].index)#xóa các dòng chứa học kỳ hè vì nó chỉ chiếm 2% trong dataset
data_merge = data_merge.reset_index()

#Divide train and test:
##80% students train and 20% students test
##recommend courses of two terms in 2016

In [ ]:
listStudent = np.unique(data_merge['mssv'])
student_train, student_test = train_test_split(listStudent, test_size=0.2, random_state=42)

data_train = data_merge[data_merge['mssv'].isin(student_train)]
data_test = data_merge[data_merge['mssv'].isin(student_test) & data_merge['namhoc'].isin([2013, 2014, 2015])]

###statistic courses in each cate of each student

In [ ]:
faculties = np.unique(b)#danh sách cách khoa
enrInFac_train = pd.DataFrame(columns = faculties)#thống kê số môn đăng ký của các khoa của từng học sinh train
enrInFac_test = pd.DataFrame(columns = faculties)#thống kê số môn đăng ký của các khoa của từng học sinh test
def a_student(dataStudent, faculties):# hàm này dùng để thống kê số đăng ký từng khoa của một học sinh
  count_enrolls = []
  static = dataStudent['KhoaMH'].value_counts()
  indexx = static.index
  for i in faculties:
    if i in indexx:
      count_enrolls.append(static[i])
    else:
      count_enrolls.append(0)
  return pd.Series(count_enrolls,index = faculties)

#Train
for i in student_train:
  newRow = a_student(data_train[data_train['mssv'] == i], faculties)
  enrInFac_train = enrInFac_train.append(newRow, ignore_index=True)
enrInFac_train.insert(0,'mssv', student_train)

#Test
for i in student_test:
  newRow = a_student(data_test[data_test['mssv'] == i], faculties)
  enrInFac_test = enrInFac_test.append(newRow, ignore_index=True)
enrInFac_test.insert(0,'mssv', student_test)


##cate of each student

In [ ]:
cateStudent = pd.DataFrame(columns=['khoa'])#đây là dữ liệu chưa khoa của từng học sinh
for i in range(listStudent.shape[0]):
  k = data_merge[data_merge['mssv'] == data_merge.loc[i,'mssv']].khoa.iloc[0]
  row = pd.Series([k], index = ['khoa'])
  cateStudent = cateStudent.append(row, ignore_index = True)
cateStudent.index = listStudent

#1. Interest Score

$Sim_m(s,s') = λSim(s,s') + (1-λ)samemajor(s,s')$
###Equation of Interest Score:
$IS(s,c) = \frac{\sum_{s' \in S_{s,k}} {I_{C_{s'}}}\times sim_m(s,s')} {\sum_{s' \in S_{s,k}} \times sim_m(s,s')}$

In [ ]:
#-----------------------------train
nml_enrInFac_train = enrInFac_train.drop('mssv', axis = 1)
nml_enrInFac_train.index = enrInFac_train['mssv']
nml_enrInFac_train = nml_enrInFac_train + 1 #---------- evade divide zero -------
#normalize by dividing max
for i in range(nml_enrInFac_train.shape[0]):
  maxx = max(nml_enrInFac_train.iloc[i])
  nml_enrInFac_train.iloc[i] = round((nml_enrInFac_train.iloc[i]/maxx).astype(float),4)

# P
norm2_train = np.linalg.norm(nml_enrInFac_train.iloc[:].astype(float), axis=1)
for i in range(norm2_train.shape[0]):
  nml_enrInFac_train.iloc[i] = nml_enrInFac_train.iloc[i]/norm2_train[i]
#student's faculty by 1. In contrast by 0
samemajor_train = pd.DataFrame(0, index=student_train, columns=faculties)
for i in range(student_train.shape[0]):
  samemajor_train.loc[student_train[i],cateStudent.loc[student_train[i],'khoa']] = 1

#-------------------------------test
nml_enrInFac_test = enrInFac_test.drop('mssv', axis = 1)
nml_enrInFac_test.index = enrInFac_test['mssv']
nml_enrInFac_test = nml_enrInFac_test + 1 #---------- evade divide zero -------
#normalize by dividing max
for i in range(nml_enrInFac_test.shape[0]): 
  maxx = max(nml_enrInFac_test.iloc[i])
  nml_enrInFac_test.iloc[i] = round((nml_enrInFac_test.iloc[i]/maxx).astype(float),4)

# P
norm2_test = np.linalg.norm(nml_enrInFac_test.iloc[:].astype(float), axis=1)
for i in range(norm2_test.shape[0]):
  norm = norm2_test[i]
  if norm == 0:
    norm = 1
  nml_enrInFac_test.iloc[i] = nml_enrInFac_test.iloc[i]/norm

#student's faculty by 1. In contrast by 0
samemajor_test = pd.DataFrame(0, index=student_test, columns=faculties)
for i in range(student_test.shape[0]):
  samemajor_test.loc[student_test[i],cateStudent.loc[student_test[i],'khoa']] = 1


lamda = 0.5 ########################chú ý thay đổi giá trị lamda cho phù hợp
k = 7

Sim = round((np.matmul(nml_enrInFac_test, nml_enrInFac_train.transpose())*lamda + np.matmul(samemajor_test, samemajor_train.transpose())*(1 - lamda)).astype(float),4)
Sim.columns = student_train

top_k = Sim.apply(lambda row: row.nlargest(k), axis=1)
top_k_mssv = pd.DataFrame(0, index = student_test,columns = [i for i in range(k)])#top-k similar student code of s as neighbors
for i in range(student_test.shape[0]):
  count = 0
  for j in range(top_k.shape[1]):
    if not pd.isna(top_k.iloc[i,j]):
      top_k_mssv.iloc[i,count] = top_k.iloc[i].index[j]
      count +=1

top_k_mssv = top_k_mssv.sort_values(by=0)

def sum_k(k, mssv, Sim, I):
  S = 0
  for i in range(k):
    S += (Sim.loc[mssv[i]]*I[i])
  return S

def check_enroll(mssv, data_train, mamh, courseCate):
  if mamh not in courseCate:
    return 0
  condition = (data_train['mssv'] == mssv) & (data_train['mamh'] == mamh)
  if condition.any():
    return 1
  return 0
student_neighbour = top_k.columns#bỏ những học sinh không thuộc neighbour
data_neighbour = data_train[data_train['mssv'].isin(student_neighbour)]
data_neighbour = data_neighbour[['mssv', 'mamh','khoa']]
courses = np.unique(data_neighbour['mamh'])

IS = pd.DataFrame(0, index = student_test, columns = courses)
I = [1 for i in range(k)]
list_course = {}
for i in faculties:
  data = data_merge[data_merge['khoa'] == i]
  list_course[i] = np.unique(data['mamh'])



for i in range(student_test.shape[0]):
  print(i)
  for j in range(courses.shape[0]):
    I_C = [check_enroll(i, data_neighbour, courses[j], list_course[cateStudent.loc[i].iloc[0]]) for i in top_k_mssv.iloc[i]]
    sim_sum1 = sum_k(k, top_k_mssv.iloc[i], Sim.iloc[i],I_C)
    sim_sum2 = sum_k(k, top_k_mssv.iloc[i], Sim.iloc[i],I)
    IS.iloc[i,j] = round(sim_sum1/sim_sum2, 4)


<ipython-input-31-5178dbd06505>:45: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  Sim = round((np.matmul(nml_enrInFac_test, nml_enrInFac_train.transpose())*lamda + np.matmul(samemajor_test, samemajor_train.transpose())*(1 - lamda)).astype(float),4)


,F451D99AXPvAibaEXe+fPSOag5leTcWuR+r937fj,6988D28AXPvAibaEXe9MSDlNDFbYdo3rYOhSopw2,7E308531XPvAibaEXe879+AOg1gh8i58Q/VMq7RU,DD10B95EXPvAibaEXe/PQrWCRBKWOUTASXEgaMpR,4CD37F46XPvAibaEXe8ge2UHv4J4Byyo6uzLKM42,CEF4F5F1XPvAibaEXe9ej2kTw4OUmtE+HmhDvEXe,AB4D8F99XPvAibaEXe/YKN5ysyYL7EjErJAJK5l+,D349E207XPvAibaEXe/aPLdUCtOgChkEtg7JE0Ca,C9CD6314XPvAibaEXe8LX4hhYKrFZr9FZE839SqA,D78141E5XPvAibaEXe9j81k8MH0oJ4zch8KMTSfQ,...,8B9DD5F1XPvAibaEXe/7VXm0xgXldIyNDzaf66fA,6B917B37XPvAibaEXe/KMtCr6bHaurGMaIHnPyOQ,C5F2150DXPvAibaEXe+tg0ae8qw/gtk+nFTaazYq,EF5725B0XPvAibaEXe+cHxM5cHw4WonybFt5QicB,479E77A5XPvAibaEXe/PGAaNNWvbUNSidEr/n3m6,496BA332XPvAibaEXe/9n4RXguPAYPE1qUnKCdpJ,554BEAD4XPvAibaEXe9hSzSdeSAT0nKSX8+l/eSA,382BCEACXPvAibaEXe8zjuWfAVBr9cQalpl5Q/IX,DE84103AXPvAibaEXe8/V1eK05z0442dkLKXGv09,CAF1ABCDXPvAibaEXe/7BYdCAEGEzTgdf8kcAw53
mssv,,,,,,,,,,,,,,,,,,,,,
4BC011B2XPvAibaEXe9iVChpJY9DITS7AYFLgUjB,0.4857,0.9804,0.4981,0.4797,0.9201,0.4662,0.3526,0.4715,0.4885,0.3758,...,0.4609,0.9446,0.4555,0.4935,0.9676,0.9297,0.9759,0.4580,0.4774,0.4552
0B431A01XPvAibaEXe87069oyWO+IS0M6kVzLXKw,0.4052,0.8878,0.4144,0.4292,0.8746,0.4339,0.2689,0.3831,0.4104,0.3020,...,0.4304,0.8847,0.4044,0.3904,0.8662,0.8922,0.8938,0.4291,0.3975,0.4099


#Timing and popularity based Score

###Xác định kỳ học của học sinh đó

In [ ]:
#Đầu tiên xác định xem sinh viên nhập học năm mấy
khoaHoc = data_merge['khoahoc'].copy()
lis = np.unique(khoaHoc)
lis_replace = np.array([2013, 2014, 2015, 2016])
khoaHoc.replace(to_replace = lis, value = lis_replace, inplace = True)
data_merge['namNhapHoc'] = khoaHoc
#Tính học kỳ của của từng đăng ký
data_merge['soHocKy'] = (data_merge['namhoc'] - data_merge['namNhapHoc'])*2 + data_merge['hocky']

###Xác định số đăng ký của các môn qua các kỳ

In [ ]:
#các kỳ học(ở đây ví dụ một sinh có 8 kỳ học trong vòng 4 năm)
semesters = ["semester_1", "semester_2", "semester_3", "semester_4", "semester_5", "semester_6", "semester_7", "semester_8"]

count_course_semester = pd.DataFrame(columns = ['mamh'] + semesters[:])#dữ liệu chứa số đăng ký của các môn học qua các kỳ
col_ccs = count_course_semester.columns#các cột của count_course_semester
numberSem = 8 # số kỳ của mỗi học sinh bằng 8
listCourse = np.unique(data_merge['mamh'])#danh sách các môn học


#hàm course_Enroll dùng để tính số đăng ký từng môn qua các kỳ
def course_Enroll(dataCourse, mamh, numberSem, col_ccs):
  c = [mamh]
  for i in range(numberSem):
    a = dataCourse[dataCourse['soHocKy'] == (i+1)]
    if a.shape[0]:
      c.append(a.shape[0])
    else: 
      c.append(0)

  return pd.Series(c, index = col_ccs)
for i in listCourse:
  row = course_Enroll(data_merge[data_merge['mamh'] == i], i, numberSem, col_ccs)
  count_course_semester = count_course_semester.append(row, ignore_index= True)

##Which semester is more suitable for taking this course?
$T_t(c,t) = \frac{Log(Num(c,t))}{Max_{t' \in T}(Log(Num(c,t')))}$ <br>
##Which courses are popular now?
$T_p(c,t) = \frac{Log(Num(c,t))}{Max_{c' \in C}(Log(Num(c',t)))}$
##The final Timing based Score:
$TS(c,t) = 2 \times \frac{T_t(c,t) \times T_p(c,t)}{T_t(c,t) + T_p(c,t)}$

In [ ]:
#calculate T_t
T_t = count_course_semester.copy()
T_t.index = count_course_semester['mamh']
T_t = T_t.drop('mamh', axis = 1)
T_t = T_t + 1#evade divide Log(0)
T_t = np.log(T_t.astype(float))
for i in range(T_t.shape[0]):
  maxx = max(T_t.iloc[i])
  T_t.iloc[i] = (T_t.iloc[i]/maxx)
#calculate T_p
T_p = count_course_semester.copy()
T_p.index = count_course_semester['mamh']
T_p = T_p.drop('mamh', axis = 1)
T_p = T_p + 1#evade divide Log(0)
T_p = np.log(T_p.astype(float))
for i in range(T_p.shape[1]):
  maxx = max(T_p[T_p.columns[i]])
  T_p.iloc[:,i] = (T_t.iloc[:,i]/maxx)

#calculate TS

TS = (T_t*T_p)/(T_t + T_p)
TS['mssv'] = TS.index
TS.to_excel('TS.xlsx', index=False)


#Grade-based Score
##To predict the grade of a course in adjacent semester for a student
$C_{S,S_{s}}$ denotes the courses that are enrolled by both students S and $S_s$

In [ ]:
#course grade of each student

courses = np.unique(data_train['mamh'])
#train
course_grade_train = pd.DataFrame(0, index = student_train, columns = courses)#điểm từng môn của của từng học sinh train
for i in range(course_grade_train.shape[0]):
  print(i)
  for j in range(course_grade_train.shape[1]):
    d = data_train[(data_train['mssv'] == course_grade_train.index[i]) & (data_train['mamh'] == course_grade_train.columns[j])]
    if not d.empty:
      course_grade_train.iloc[i,j] = d.loc[d.index[0], 'diem']
#test
course_grade_test = pd.DataFrame(0, index = student_test, columns = courses)#điểm từng môn của của từng học sinh test
for i in range(course_grade_test.shape[0]):
  print(i)
  for j in range(course_grade_test.shape[1]):
    d = data_test[(data_test['mssv'] == course_grade_test.index[i]) & (data_test['mamh'] == course_grade_test.columns[j])]
    if not d.empty:
      course_grade_test.iloc[i,j] = d.loc[d.index[0], 'diem']


###GPA of each student in per semester

In [ ]:
#Tính số chỉ từng kỳ của
terms = ["2013_1", "2013_2", "2014_1", "2014_2", "2015_1", "2015_2", "2016_1", "2016_2"]#các kỳ học
years = [2013, 2014, 2015, 2016]
ectsOfTerms = pd.DataFrame(columns = terms)#số tín chỉ theo của kỳ của từng học sinh

#hàm này dùng để tính số tín chỉ của từng học sinh
def ects(dataStudent, years, terms):
  e = []
  for i in years:
    a = dataStudent[(dataStudent['namhoc'] == i) & (dataStudent['hocky'] == 1)]
    b = dataStudent[(dataStudent['namhoc'] == i) & (dataStudent['hocky'] == 2)]
    if a.shape[0]:
      e.append(sum(a['sotc']))
    else: 
      e.append(0)
    if b.shape[0]:
      e.append(sum(b['sotc']))
    else: 
      e.append(0)
  return pd.Series(e, index = terms)

for i in listStudent:
  row = ects(data_merge[data_merge['mssv'] == i], years, terms)
  ectsOfTerms = ectsOfTerms.append(row, ignore_index=True)
ectsOfTerms.insert(0,'mssv', listStudent)

In [ ]:
gpaTerms = pd.DataFrame(columns = terms)#điểm trung bình của từng học sinh theo cách kỳ học

#hàm này dùng để tính điểm trung bình từng kỳ của các học sinh
def gpa(dataStudent, years, terms, ectsStudent):
#ectsStudent là số tín chỉ các kỳ của một học sinh s
  g = []
  Sum = 0
  for i in range(len(years)):
    a = dataStudent[(dataStudent['namhoc'] == years[i]) & (dataStudent['hocky'] == 1)]
    b = dataStudent[(dataStudent['namhoc'] == years[i]) & (dataStudent['hocky'] == 2)]
    #kỳ 1
    if a.shape[0]:
      Sum += sum(a['sotc']*a['diem'])
      g.append(round(Sum/sum(ectsStudent.iloc[0,1:i*2 + 2]),2))
    else: 
      g.append(round(Sum/sum(ectsStudent.iloc[0,1:i*2 + 2]),2))
    #kỳ 2
    if b.shape[0]:
      Sum += sum(b['sotc']*b['diem'])
      g.append(round(Sum/sum(ectsStudent.iloc[0,1:i*2 + 3]),2))
    else: 
      g.append(round(Sum/sum(ectsStudent.iloc[0,1:i*2 + 3]),2))
  return pd.Series(g, index = terms)

for i in listStudent:
  row = gpa(data_merge[data_merge['mssv'] == i], years, terms, ectsOfTerms[ectsOfTerms['mssv'] == i])
  gpaTerms = gpaTerms.append(row, ignore_index=True)
gpaTerms.insert(0,'mssv', listStudent)


##GPA of student for train and test

In [ ]:
#train
gpaTerms_train = gpaTerms[gpaTerms['mssv'].isin(student_train)]
gpaTerms_train.index = gpaTerms_train['mssv']
gpaTerms_train = gpaTerms_train.drop('mssv', axis = 1)
gpa_train = pd.DataFrame(0, index = student_train, columns = ['average'])

for i in range(student_train.shape[0]):
  gpa_train.iloc[i,0] = gpaTerms_train.loc[student_train[i]].mean(skipna=True)

#test
gpaTerms_test = gpaTerms[gpaTerms['mssv'].isin(student_test)]
gpaTerms_test.index = gpaTerms_test['mssv']
gpaTerms_test = gpaTerms_test.drop('mssv', axis = 1)
gpa_test = pd.DataFrame(0, index = student_test, columns = ['average'])

for i in range(student_test.shape[0]):
  gpa_test.iloc[i,0] = gpaTerms_test.loc[student_test[i],:'2015_2'].mean(skipna=True)
gpa_test = gpa_test.dropna()

#just can predict grade of students who had previour grade
#Therefore I remove freshmans 

In [ ]:
course_grade_test = pd.read_excel('course_grade_test.xlsx')
course_grade_train = pd.read_excel('course_grade_train.xlsx')

In [ ]:
student_predict = gpa_test.index # danh sách cách học sinh có thể dự đoán điểm vì có dữ liệu điểm từ kỳ trc
course_grade_test = course_grade_test[course_grade_test['mssv'].isin(student_predict)]

$sim(S,S_s) = \frac{\sum_{i \in C_{S,S_{s}}}(g_{si} - mean(g_{si}))(g_{s_{s}i} - mean(g_{s_{s}i}))} {\sqrt{\sum_{i \in C_{S,S_{s}}}(g_{si} - mean(g_{si}))^2} \sqrt{\sum_{i \in C_{S,S_{s}}}(g_{s_{s}i} - mean(g_{s_{s}i}))^2}}$

In [ ]:
course_grade_train.index = course_grade_train['mssv']
course_grade_train = course_grade_train.drop('mssv', axis = 1)#xóa côt mssv
course_grade_test.index = course_grade_test['mssv']
course_grade_test = course_grade_test.drop('mssv', axis = 1)

In [ ]:
Sim_grade = pd.DataFrame(0, index = student_predict, columns = student_train)
for i in range(Sim_grade.shape[0]):
  print(i)
  for j in range(Sim_grade.shape[1]):
    a = course_grade_test.loc[Sim_grade.index[i], :]*course_grade_train.loc[Sim_grade.columns[j], :]
    common = a.index[a != 0 ].tolist()
    if common:
      g_si = course_grade_test.loc[Sim_grade.index[i]] - gpa_test.loc[Sim_grade.index[i]].iloc[0]
      g_ssi = course_grade_train.loc[Sim_grade.columns[j]] - gpa_train.loc[Sim_grade.columns[j]].iloc[0]
      numerator = sum([(g_si.loc[t]*g_ssi.loc[t]) for t in common])
      denominator = np.sqrt(sum([(g_si.loc[t])**2 for t in common])) * np.sqrt(sum([(g_ssi.loc[t])**2 for t in common]))
      Sim_grade.iloc[i,j] = numerator/denominator

In [ ]:
Sim_grade = pd.read_excel('Sim_grade.xlsx')
Sim_grade.index = Sim_grade['mssv']
Sim_grade = Sim_grade.drop('mssv', axis = 1)

In [ ]:
####top-k trong Sim_grade
k = 7

grade_k = Sim_grade.apply(lambda row: row.nlargest(k), axis=1)
grade_k_mssv = pd.DataFrame(0, index = grade_k.index,columns = [i for i in range(k)])#top-k similar student code of s as neighbors
for i in range(grade_k.shape[0]):
  count = 0
  for j in range(grade_k.shape[1]):
    if not pd.isna(grade_k.loc[grade_k.index[i], grade_k.columns[j]]):
      grade_k_mssv.iloc[i,count] = grade_k.columns[j]
      count +=1



In [ ]:
####Điểm trung bình của từng môn học
avg_courses = pd.DataFrame(0, index = ['average'], columns = course_grade_train.columns)
numStudent = course_grade_train.shape[0] + course_grade_test.shape[0]
for i in range(avg_courses.shape[1]):

  Sum_train = sum(course_grade_train.loc[:,avg_courses.columns[i]])
  student0_train = course_grade_train[avg_courses.columns[i]].value_counts()[0.0] # những học sinh không có dữ liệu trong kỳ đó

  Sum_test = sum(course_grade_test.loc[:,avg_courses.columns[i]])
  student0_test = course_grade_test[avg_courses.columns[i]].value_counts()[0.0]
  
  avg_courses.iloc[0,i] = (Sum_train + Sum_test)/(numStudent - student0_train - student0_test)


<ipython-input-57-557d9a5c60f2>:12: RuntimeWarning: invalid value encountered in long_scalars
  avg_courses.iloc[0,i] = (Sum_train + Sum_test)/(numStudent - student0_train - student0_test)
<ipython-input-57-557d9a5c60f2>:12: RuntimeWarning: invalid value encountered in long_scalars
  avg_courses.iloc[0,i] = (Sum_train + Sum_test)/(numStudent - student0_train - student0_test)
<ipython-input-57-557d9a5c60f2>:12: RuntimeWarning: invalid value encountered in long_scalars
  avg_courses.iloc[0,i] = (Sum_train + Sum_test)/(numStudent - student0_train - student0_test)


7.697297263789821

###Điểm trung bình của tất cả các khóa học của tất cả các học sinh trong một kỳ

In [ ]:
gpaTerms2 = gpaTerms.copy()
gpaTerms2.replace(to_replace = np.nan, value = 0, inplace =True)

# avgAll điểm trung bình của tất cả các học sinh trong một kỳ là
avgAll_semester = []
numberStudents = len(listStudent)
for i in range(len(terms)):

  Sum = sum(gpaTerms2[terms[i]])
  student0 = gpaTerms2[terms[i]].value_counts()[0.0] # những học sinh không có dữ liệu trong kỳ đó
  avgAll_semester.append(round(Sum/(numberStudents-student0),2))

avgAll = sum(avgAll_semester)/len(avgAll_semester)


In [ ]:
####top-k trong Sim_grade(grade_k_mssv)
####Điểm trung bình của từng học sinh(gpa_train, gpa_test)
####Điểm trung bình của từng môn học(avg_courses)
####điểm trung bình tất cả(avgAll)

$g_{sc} = b_{sc} + \frac {\sum_{S_s \in S_{s,k}}(g_{S_{s}C} - b_{S_{s}C}) \times sim(S,S_{s})} {\sum_{S_{s} \in S_{s,k}}sim(S,S_s)}$

In [ ]:
##compute b_sc
##b_sc = GPA(s) + average(c) - avgAll
b_sc_train = pd.DataFrame(0, index = course_grade_train.index, columns = course_grade_train.columns)

for i in range(b_sc_train.shape[0]):
  for j in range(b_sc_train.shape[1]):
    # if course_grade_train.loc[b_sc_train.index[i], b_sc_train.columns[j]]:# nếu môn nào học sinh không tham gia thì không phải tính độ chênh lệch
      b_sc_train.iloc[i,j] = gpa_train.loc[b_sc_train.index[i], 'average'] + avg_courses.loc[avg_courses.index[0], b_sc_train.columns[j]] - avgAll

b_sc_test = pd.DataFrame(0, index = course_grade_test.index, columns = course_grade_test.columns)

for i in range(b_sc_test.shape[0]):
  for j in range(b_sc_test.shape[1]):
    # if course_grade_test.loc[b_sc_test.index[i], b_sc_test.columns[j]]:# nếu môn nào học sinh không tham gia thì không phải tính độ chênh lệch
      b_sc_test.iloc[i,j] = gpa_test.loc[b_sc_test.index[i], 'average'] + avg_courses.loc[avg_courses.index[0], b_sc_test.columns[j]] - avgAll

In [ ]:
# compute g_sc
g_sc = pd.DataFrame(0, index = course_grade_test.index, columns = course_grade_test.columns)
for i in range(g_sc.shape[0]):
  print(i)
  for j in range(g_sc.shape[1]):
    numerator = sum([(course_grade_train.loc[t, g_sc.columns[j]] - b_sc_train.loc[t, g_sc.columns[j]])*(Sim_grade.loc[g_sc.index[i], t]) for t in grade_k_mssv.loc[g_sc.index[i]]])
    denominator = sum([(Sim_grade.loc[g_sc.index[i], t]) for t in grade_k_mssv.loc[g_sc.index[i]]])
    g_sc.iloc[i,j] = b_sc_test.loc[g_sc.index[i], g_sc.columns[j]] + numerator/denominator


In [ ]:
for i in range(g_sc.shape[0]):
  g_sc.iloc[i] = g_sc.iloc[i]/(max(g_sc.iloc[i]))

In [ ]:
g_sc2 = pd.DataFrame(0, index = course_grade_test.index, columns = course_grade_test.columns)
for i in range(g_sc2.shape[0]):
  print(i)
  for j in range(g_sc2.shape[1]):
    numerator = sum([(course_grade_train.loc[t, g_sc2.columns[j]])*(Sim_grade.loc[g_sc2.index[i], t]) for t in grade_k_mssv.loc[g_sc2.index[i]]])
    denominator = sum([(Sim_grade.loc[g_sc2.index[i], t]) for t in grade_k_mssv.loc[g_sc2.index[i]]])
    g_sc2.iloc[i,j] = numerator/denominator
    

In [ ]:
g_sc['mssv'] = g_sc.index
g_sc2['mssv'] = g_sc2.index


<ipython-input-105-2a1c1344de99>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  g_sc2['mssv'] = g_sc2.index


In [ ]:
g_sc.to_excel('GS.xlsx', index = False)
g_sc2.to_excel('GS_replace.xlsx', index = False)

###Điểm trung bình một môn trong một học kỳ

In [ ]:
# courseTerms Điểm trung bình một môn trong một học kỳ
courseTerms = pd.DataFrame(columns = ['mamh'] + terms[:])
listCourse = np.unique(data_merge['mamh'])#danh sách các môn học
courseTerms_column = courseTerms.columns
def avgCourse(dataCourse, terms, mamh, years, courseTerms_column):
  av = [mamh]
  Sum = 0
  Count = 1 #Biến đếm số đăng ký của môn cho bằng 1 vì để chánh chia cho 0
  for i in range(len(years)):
    a = dataCourse[(dataCourse['namhoc'] == years[i]) & (dataCourse['hocky'] == 1)]
    b = dataCourse[(dataCourse['namhoc'] == years[i]) & (dataCourse['hocky'] == 2)]
    if a.shape[0]:
      Sum += sum(a['diem'])
      Count += a.shape[0]
      av.append(round(Sum/Count,2))
    else:
      av.append(round(Sum/Count,2))
    if b.shape[0]:
      Sum += sum(b['diem'])
      Count += b.shape[0]
      av.append(round(Sum/Count,2))
    else:
      av.append(round(Sum/Count,2))
  return pd.Series(av, index = courseTerms_column)
for i in listCourse:
  row = avgCourse(data_merge[data_merge['mamh'] == i], terms, i, years, courseTerms_column)
  courseTerms = courseTerms.append(row, ignore_index=True)

###DataFrame khoa của từng học sinh

In [ ]:
cateStudent = pd.DataFrame(columns=['mssv','khoa'])#đây là dữ liệu chưa khoa của từng học sinh
for i in range(listStudent.shape[0]):
  k = data_merge[data_merge['mssv'] == data_merge.loc[i,'mssv']].khoa.iloc[0]
  row = pd.Series([listStudent[i], k], index = ['mssv', 'khoa'])
  cateStudent = cateStudent.append(row, ignore_index = True)



#Trong project có 3 feature dùng để đánh giá có nên đăng ký 1 môn học cho kỳ tiếp theo không:
##1 Interest Score gồm có:
### số môn đăng ký của từng khoa của từng học sinh(enrInFac)
### khoa của từng học sinh(cateStudent)
##2 Timing and popularity based Score gồm có:
###Xác định số đăng ký của các môn qua các kỳ(count_course_semester)
##3 Grade Score gồm có
###điểm trung bình của học sinh theo từng kỳ(gpaTerms)
###Điểm trung bình của tất cả các khóa học của tất cả các học sinh trong một kỳ(avgAll)
###Điểm trung bình một môn trong một học kỳ(courseTerms)

#Tổng cộng có 4 năm 8 kỳ vì vậy lấy 3 năm đầu làm tập train còn năm cuối cùng làm tập test

In [ ]:
from sklearn.metrics import f1_score

# Danh sách môn học dự đoán
y_pred = ["Math", "Physics", "Chemistry", "Biology"]

# Danh sách môn học thực tế
y_true = ["Math", "Physics", "English", "Biology"]

# Tính toán F1 score
f1 = f1_score(y_true, y_pred, average='weighted')

# In kết quả
print("F1 score:", f1)


F1 score: 0.75
